In [15]:
import numpy as np
import xarray as xr
from importlib import reload

from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import derive_potential_density_values_TEST
from analysis_package import ecco_masks
from analysis_package import interpolation

interpolation = reload(interpolation)
ecco_masks = reload(ecco_masks)
plotting_functions = reload(plotting_functions)
open_datasets = reload(open_datasets)
derive_potential_density_values_TEST = reload(derive_potential_density_values_TEST)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
data_dir = "./nctiles_monthly/"

# PHIHYD: insitu pressure anomaly with respect to the depth integral of gravity and reference density (g*rho_reference)
PHIHYD_var_str = "PHIHYD"
# SALT: insitu salinity (psu)
SALT_var_str = "SALT"
# THETA: potential pressure (C)
THETA_var_str = "THETA"

######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################

grid_path = "./ecco_grid/ECCOv4r3_grid.nc"
grid = xr.open_dataset(grid_path)

In [4]:
all_times_times_slice = np.arange(0,288)

P_INSITU_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, "P_INSITU", all_times_times_slice,rename_indices=False)
T_INSITU_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, "T_INSITU", all_times_times_slice,rename_indices=False)
SALT_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, "SALT", all_times_times_slice,rename_indices=True)

tile_coords = np.arange(0,13)
P_INSITU_ds = P_INSITU_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50)).assign_coords(tile=tile_coords)
T_INSITU_ds = T_INSITU_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50)).assign_coords(tile=tile_coords)
SALT_ds = SALT_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50)).assign_coords(tile=tile_coords)


Loaded P_INSITU over time slice  

Loaded T_INSITU over time slice  

Loaded SALT over time slice  



In [18]:
print(P_INSITU_ds)

<xarray.Dataset>
Dimensions:   (i: 90, j: 90, k: 50, tile: 13, time: 288)
Coordinates:
  * time      (time) float64 1.0 2.0 3.0 4.0 5.0 ... 285.0 286.0 287.0 288.0
  * i         (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    tim       (time) datetime64[ns] 1992-01-16 1992-02-16 ... 2015-12-16
  * j         (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    dep       (k) float64 5.0 15.0 25.0 35.0 ... 5.039e+03 5.461e+03 5.906e+03
  * k         (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
    lat       (tile, j, i) float64 -88.24 -88.38 -88.52 ... -88.03 -88.08 -88.1
    lon       (tile, j, i) float64 -111.6 -111.3 -110.9 ... -99.42 -105.6 -111.9
  * tile      (tile) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    P_INSITU  (tile, time, k, j, i) float64 nan nan nan nan ... nan nan nan nan
    timstep   (tile, time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    land      (tile, k, j, i) float64 nan nan nan

In [5]:
# This script is for deriving these state values given ECCO variables


"""
for i in range(1,14):
    PDENS_ds, P_INSITU_ds, T_ISITU_ds = derive_potential_density_values_TEST.make_potential_density_dataset(PHIHYD_ds_raw.isel(tile=i-1), 
                                                                                            SALT_ds_raw.isel(tile=i-1), 
                                                                                            THETA_ds_raw.isel(tile=i-1), 
                                                                                            all_times_times_slice, 
                                                                                            ref_pressure=2000.)
    if i < 10:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")
        P_INSITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU/P_INSITU.000"+str(i)+".nc")
        print("saved P_INSITU.000"+str(i)+".nc")
        T_ISITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU/T_INSITU.000"+str(i)+".nc")
        print("saved T_INSITU.000"+str(i)+".nc")
    else:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.00"+str(i)+".nc")
        P_INSITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU/P_INSITU.00"+str(i)+".nc")
        print("saved P_INSITU.00"+str(i)+".nc")
        T_ISITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU/T_INSITU.00"+str(i)+".nc")
        print("saved T_INSITU.00"+str(i)+".nc")

    print("finished "+str(i))
"""

'\nfor i in range(1,14):\n    PDENS_ds, P_INSITU_ds, T_ISITU_ds = derive_potential_density_values_TEST.make_potential_density_dataset(PHIHYD_ds_raw.isel(tile=i-1), \n                                                                                            SALT_ds_raw.isel(tile=i-1), \n                                                                                            THETA_ds_raw.isel(tile=i-1), \n                                                                                            all_times_times_slice, \n                                                                                            ref_pressure=2000.)\n    if i < 10:\n        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")\n        P_INSITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU/P_INSITU.000"+str(i)+".nc")\n        print("saved P_INSITU.000"+str(i)+".nc")\n        T_ISITU_ds.to_netcdf("../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly

In [19]:
P_INSITU_at_u_points, slope, j = interpolation.interpolate_c_to_u(P_INSITU_ds.P_INSITU.isel(k=slice(0,1)),grid)
P_INSITU_at_v_points, slope, j = interpolation.interpolate_c_to_v(P_INSITU_ds.P_INSITU.isel(k=slice(0,1)),grid)

checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5


In [28]:
for i in range(1,14):

    if i < 10:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")
        P_INSITU_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU_U_surfacelvl/P_INSITU_U_surfacelvl.000"+str(i)+".nc")
        print("saved P_INSITU_U_surfacelvl.000"+str(i)+".nc")
        P_INSITU_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU_V_surfacelvl/P_INSITU_V_surfacelvl.000"+str(i)+".nc")        
        print("saved P_INSITU_V_surfacelvl.000"+str(i)+".nc")
    else:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.00"+str(i)+".nc")
        P_INSITU_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU_U_surfacelvl/P_INSITU_U_surfacelvl.00"+str(i)+".nc")
        print("saved P_INSITU_U_surfacelvl.00"+str(i)+".nc")
        P_INSITU_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/P_INSITU_V_surfacelvl/P_INSITU_V_surfacelvl.00"+str(i)+".nc")
        print("saved P_INSITU_V_surfacelvl.00"+str(i)+".nc")

saved P_INSITU_U_surfacelvl.0001.nc
saved P_INSITU_V_surfacelvl.0001.nc
saved P_INSITU_U_surfacelvl.0002.nc
saved P_INSITU_V_surfacelvl.0002.nc
saved P_INSITU_U_surfacelvl.0003.nc
saved P_INSITU_V_surfacelvl.0003.nc
saved P_INSITU_U_surfacelvl.0004.nc
saved P_INSITU_V_surfacelvl.0004.nc
saved P_INSITU_U_surfacelvl.0005.nc
saved P_INSITU_V_surfacelvl.0005.nc
saved P_INSITU_U_surfacelvl.0006.nc
saved P_INSITU_V_surfacelvl.0006.nc
saved P_INSITU_U_surfacelvl.0007.nc
saved P_INSITU_V_surfacelvl.0007.nc
saved P_INSITU_U_surfacelvl.0008.nc
saved P_INSITU_V_surfacelvl.0008.nc
saved P_INSITU_U_surfacelvl.0009.nc
saved P_INSITU_V_surfacelvl.0009.nc
saved P_INSITU_U_surfacelvl.0010.nc
saved P_INSITU_V_surfacelvl.0010.nc
saved P_INSITU_U_surfacelvl.0011.nc
saved P_INSITU_V_surfacelvl.0011.nc
saved P_INSITU_U_surfacelvl.0012.nc
saved P_INSITU_V_surfacelvl.0012.nc
saved P_INSITU_U_surfacelvl.0013.nc
saved P_INSITU_V_surfacelvl.0013.nc


In [29]:
T_INSITU_at_u_points, slope, j = interpolation.interpolate_c_to_u(T_INSITU_ds.TEMP_INSITU.isel(k=slice(0,1)),grid)
T_INSITU_at_v_points, slope, j = interpolation.interpolate_c_to_v(T_INSITU_ds.TEMP_INSITU.isel(k=slice(0,1)),grid)

for i in range(1,14):
    if i < 10:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")
        T_INSITU_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU_U_surfacelvl/T_INSITU_U_surfacelvl.000"+str(i)+".nc")
        print("saved T_INSITU_U_surfacelvl.000"+str(i)+".nc")
        T_INSITU_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU_V_surfacelvl/T_INSITU_V_surfacelvl.000"+str(i)+".nc")        
        print("saved T_INSITU_V_surfacelvl.000"+str(i)+".nc")
    else:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.00"+str(i)+".nc")
        T_INSITU_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU_U_surfacelvl/T_INSITU_U_surfacelvl.00"+str(i)+".nc")
        print("saved T_INSITU_U_surfacelvl.00"+str(i)+".nc")
        T_INSITU_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/T_INSITU_V_surfacelvl/T_INSITU_V_surfacelvl.00"+str(i)+".nc")
        print("saved T_INSITU_V_surfacelvl.00"+str(i)+".nc")
        


checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
saved T_INSITU_U_surfacelvl.0001.nc
saved T_INSITU_V_surfacelvl.0001.nc
saved T_INSITU_U_surfacelvl.0002.nc
saved T_INSITU_V_surfacelvl.0002.nc
saved T_INSITU_U_surfacelvl.0003.nc
saved T_INSITU_V_surfacelvl.0003.nc
saved T_INSITU_U_surfacelvl.0004.nc
saved T_INSITU_V_surfacelvl.0004.nc
saved T_INSITU_U_surfacelvl.0005.nc
saved T_INSITU_V_surfacelvl.0005.nc
saved T_INSITU_U_surfacelvl.0006.nc
saved T_INSITU_V_surfacelvl.0006.nc
saved T_INSITU_U_surfacelvl.0007.nc
saved T_INSITU_V_surfacelvl.0007.nc
saved T_INSITU_U_surfacelvl.0008.nc
saved T_INSITU_V_surfacelvl.0008.nc
saved T_INSITU_U_surfacelvl.0009.nc
saved T_INSITU_V_surfacelvl.0009.nc
saved T_INSITU_U_surfacelvl.0010.nc
saved T_INSITU_V_surfacelvl.0010.nc
saved T_INSITU_U_surfacelvl.0011.nc
saved T_INSITU_V_surfacelvl.0011.nc
saved T_INSITU_U_surfacelvl.0012.nc
saved T_INSITU_V_surfacelvl.0012.nc
saved 

In [30]:
SALT_at_u_points, slope, j = interpolation.interpolate_c_to_u(SALT_ds.SALT.isel(k=slice(0,1)),grid)
SALT_at_v_points, slope, j = interpolation.interpolate_c_to_v(SALT_ds.SALT.isel(k=slice(0,1)),grid)

for i in range(1,14):
    if i < 10:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")
        SALT_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/SALT_U_surfacelvl/SALT_U_surfacelvl.000"+str(i)+".nc")
        print("saved SALT_U_surfacelvl.000"+str(i)+".nc")
        SALT_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/SALT_V_surfacelvl/SALT_V_surfacelvl.000"+str(i)+".nc")        
        print("saved SALT_V_surfacelvl.000"+str(i)+".nc")
    else:
        #PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.00"+str(i)+".nc")
        SALT_at_u_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/SALT_U_surfacelvl/SALT_U_surfacelvl.00"+str(i)+".nc")
        print("saved SALT_U_surfacelvl.00"+str(i)+".nc")
        T_INSITU_at_v_points.isel(tile=i-1,k=0).to_netcdf("../../../Datasets-For-Projects/ECCO_Datasets/nctiles_monthly/SALT_V_surfacelvl/SALT_V_surfacelvl.00"+str(i)+".nc")
        print("saved SALT_V_surfacelvl.00"+str(i)+".nc")


checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
saved SALT_U_surfacelvl.0001.nc
saved SALT_V_surfacelvl.0001.nc
saved SALT_U_surfacelvl.0002.nc
saved SALT_V_surfacelvl.0002.nc
saved SALT_U_surfacelvl.0003.nc
saved SALT_V_surfacelvl.0003.nc
saved SALT_U_surfacelvl.0004.nc
saved SALT_V_surfacelvl.0004.nc
saved SALT_U_surfacelvl.0005.nc
saved SALT_V_surfacelvl.0005.nc
saved SALT_U_surfacelvl.0006.nc
saved SALT_V_surfacelvl.0006.nc
saved SALT_U_surfacelvl.0007.nc
saved SALT_V_surfacelvl.0007.nc
saved SALT_U_surfacelvl.0008.nc
saved SALT_V_surfacelvl.0008.nc
saved SALT_U_surfacelvl.0009.nc
saved SALT_V_surfacelvl.0009.nc
saved SALT_U_surfacelvl.0010.nc
saved SALT_V_surfacelvl.0010.nc
saved SALT_U_surfacelvl.0011.nc
saved SALT_V_surfacelvl.0011.nc
saved SALT_U_surfacelvl.0012.nc
saved SALT_V_surfacelvl.0012.nc
saved SALT_U_surfacelvl.0013.nc
saved SALT_V_surfacelvl.0013.nc
